In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.insert(0,os.path.expandvars('$CODE_MEMORY_ERRORS'))

import config2
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'00
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
from config2 import *

#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'figure')

from dateutil import parser
from error_sensitivity import getAnalysisVarnames

print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

import figure.plots

# sliding dec err-sens

In [ ]:
!ls /p/project/icei-hbp-2020-0012/lyon/memerr/data2/sub01_WGPOZPEE/results/
#corr_spoc_es_sliding2_no_filter
#corrr_spoc_es_sliding2_no_filter

In [ ]:
#from .plots import *

In [ ]:
list(freq_name2freq.keys() )

In [ ]:

#getAnalysisVarnames?

In [ ]:
getAnalysisVarnames(time_locked, control_type)

In [ ]:
-0.5 - 0.464 * 10, -0.5 + 0.464 * 12

# Plot sliding

In [ ]:
gc.collect()

In [ ]:
# collect
import gc; gc.collect()
plot_diff_vs_es = 0
plot_diff = 0

regression_type = 'Ridge'
time_lockeds = ['target', 'feedback']
#freq_names = ['broad']
freq_names = ['broad', 'beta', 'gamma']
control_types = [ 'movement', 'feedback']
#subject_inds = [15,16,17,18,19] # up to 10 fine
subject_inds = np.arange(len(subjects))
#subject_inds = list(range(12)) + list(range(13,20))
#subject_inds = [12]

# good combination to plot robust = 1, discard twice = 0,1
#time_start = parser.parse('00:00 of 2nd june 2023'); 
#time_end = parser.parse('16:03 of 3rd june 2023'); 

time_start = parser.parse('14:28 of 14th june 2023'); 
#time_end = parser.parse('16:03 of 3rd june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 

custom_prefix = None

use_preload_df = 0
load_df        = 0
exit_after = 'collect'
#exit_after = 'end'
plot_scores_per_subj = 0
ktes = ['err_sens','correction','error', 'prev_error' ]#[:2]
varnames = ktes
tmin_desired = None
require_all_subjects = 0

import traceback
DEBUG = 0
save_df = 0

#%debug
failed = []
dfs = []
for hpass in ['no_filter']: #,'0.1']:
    for env in [['stable','random']]:
        #for freq_name in list(freq_name2freq.keys() ):
        for freq_name in freq_names:
            for time_locked in time_lockeds:
                for control_type in control_types:                    
                    df = None
                    s=  ','.join( getAnalysisVarnames(time_locked, control_type)[0] )
                    varnames_b2b = [s, 'err_sens,prev_error,error']

                    output_folder = f'corr_spoc_es_sliding2_{hpass}'        
                    script_name = pjoin(scripts_dir,
                            'correlation_error_sensitivity_spoc_decoding_sliding.py')
                    try:
                        ipy.run_line_magic('run','-i {script_name}')                  
                        dfs += [df]                        
#                         if not DEBUG:
#                             del df
                    except Exception as e:                    
                        print('EXC',e)
                        exc_info = sys.exc_info()
                        exc = traceback.TracebackException(*exc_info, capture_locals=True)
                        ei = exc_info[2]    
                        #display(ei.tb_frame)
                        psf = ei

                        lfprev = None; lf = None
                        lfs = []
                        di = 0
                        while psf.tb_frame is not None:
                            stackframe = psf.tb_frame
                            display(di, stackframe)

                            psf = psf.tb_next  
                            if 'conda' not in stackframe.f_code.co_filename:        
                                lfprev = lf
                                lf = stackframe.f_locals
                                lfs += [lf]
                                di += 1
                            if psf is None:
                                break    

                        #raise e

                        print('Exc for ', hpass,freq_name)
                        failed += [ (hpass, freq_name)]
                        print('EXC',e)
                    #df.to_pickle( pjoin(path_fig, save_folder, 'df') )                                        
                    import gc; gc.collect()
                    plt.close()
            #break
        #break
    #break
print('Collection finished')

dfc =pd.concat(dfs)

dfc_allcollected = dfc

catcols = ['subject', 'fn', 'fn_full', 'SLURM_job_id', 'env',
       'trial_group_col_calc', 'rt', 'time_locked', 'control_type',
       'custom_suffix', 'freq_name', 'tmin', 'tmax', 'dec_type']
for col in catcols:
    dfc[col] = dfc[col].astype('category')

dfc['jobind'] = dfc['SLURM_job_id'].apply(lambda x: int(x.split('_')[1] ) )

In [ ]:
dfc.duplicated(catcols).sum()

In [ ]:
print(dfc['jobind'].astype('int').max())

In [ ]:
  #won't work if columns contain composite types
#dfc.duplicated()

# print( dfc['time_locked'].unique() )
# for dfi,df in enumerate(dfs):
#     print(dfi)
#     if df is None:
#         continue
#     print(df.duplicated(catcols).sum() , len(df), df['time_locked'].unique() )

In [ ]:
dfc[catcols].describe(include='all').loc['unique', :]

#### dbg

In [ ]:
tmin_desired

In [ ]:
time_lockeds

In [ ]:
time_locked

In [ ]:
try:
    collectDecodingResults(subjects, subject_inds[:1], output_folder,freq_name, 
                    env, control_type, 
                    time_locked, custom_prefix,                       
                     tmin_desired,                                                                                                                             
                     varnames, varnames_b2b,                                                                                                                   
                     time_start, time_end, verbose=1) 
    
except Exception as e:                    
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]    
    #display(ei.tb_frame)
    psf = ei

    lfprev = None; lf = None
    lfs = []
    di = 0
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame        

        psf = psf.tb_next  
        if ('conda' not in stackframe.f_code.co_filename) and\
            ('mamba' not in stackframe.f_code.co_filename):        
            display(di, stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            di += 1
        if psf is None:
            break 
        
    raise e

In [ ]:
lf['order']

In [ ]:
type( lf['order'] )

In [ ]:
lf['df_to_set']['prev_movement,prev_error,error,prev_belief_varnames']

In [ ]:
 lf['kvv'] ,  lf['index'] , lf['key']#, lf['df_to_set'][lf['index'] , lf['key']]

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df.at[0, 'A'] = list(np.array([5, 6]))

In [ ]:
Length of values (315) does not match length of index (264)

In [ ]:
lf['df_to_set'][lf['index'] , lf['key']] = lf['kvv'].tolist()

In [ ]:
varnames_b2b

#### check collected

In [ ]:
dec_types_found = dfc_allcollected['dec_type'].unique()
print('dec_types_found = ',dec_types_found)
assert len(dec_types_found) > 1

def calcTminSetVariability(df_, cols):
    r = dfc.groupby(cols)['tmin'].nunique()#.apply(lambda x: len(x) )
    return r.to_frame().describe().loc['std'].values[0]

In [ ]:
#dfc.groupby(cols_unique).size().max()

In [ ]:
import pandas as pd

# Define a function to find the minimal set of categorical columns
def find_minimal_columns(dfc):
  # Get the list of categorical columns in the DataFrame
  cat_cols = dfc.select_dtypes(include='category').columns.tolist()
  # Initialize an empty list to store the selected columns
  selected_cols = []
  # Initialize a variable to store the number of duplicates
  num_duplicates = dfc.duplicated().sum()
  # Loop through the categorical columns
  for col in cat_cols:
    # Add the current column to the selected columns
    selected_cols.append(col)
    # Create a new DataFrame with only the selected columns
    dfc_new = dfc[selected_cols]
    # Calculate the number of duplicates in the new DataFrame
    num_duplicates_new = dfc_new.duplicated().sum()
    # If the number of duplicates is zero, break the loop
    if num_duplicates_new == 0:
        break
    # Otherwise, update the number of duplicates
    else:
        num_duplicates = num_duplicates_new
  # Return the selected columns
  return selected_cols

# Call the function on dfc and print the result
minimal_columns = find_minimal_columns(dfc)
print('The minimal set of categorical columns is:', minimal_columns)

In [ ]:
#dfc.describe?

In [ ]:
# make it hang
#dfc.describe(include='all').loc['unique', :]

In [ ]:
#dfc.columns

In [ ]:
print(dfc['mtime'].min(), dfc['mtime'].max())

cols_unique = ['custom_suffix', 'trial_group_col_calc', 
    'time_locked','control_type', 
    'freq_name', 'dec_type', 'env','subject','tmin']
if dfc.duplicated(cols_unique).any():
    print('DUPLICATES:')
    display( dfc[ dfc.duplicated(cols_unique) ] )
else:
    print('No duplicates (tmin + custom_prefix)!')

print( len(dfc), dfc['subject'].nunique(), dfc['control_type'].nunique(), subject_inds )


cols_unique_ = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'tmin', 'dec_type']
if dfc.duplicated(cols_unique_).any():
    sz =  dfc.groupby(cols_unique_).size()
    print('Within suffix (fixed tmin): min dup = {}, max dup = {}'.\
                format(sz.min(), sz.max() ) )
else:
    print('No duplicates for same time across suffixes!')
    
cols_unique_2 = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'dec_type', 'custom_suffix']
sz =  dfc.groupby(cols_unique_2).size()
# if std of len is > 0, then we might need be in situation where there were two calc runs 
# with different tmins set
print('Within tmin (fixed suffix): min dup = {}, max dup = {}. Std of len = {:.2f}'.\
            format(sz.min(), sz.max(),  calcTminSetVariability(dfc, cols_unique_2) ) )
    
grp = dfc.groupby(['custom_suffix','time_locked'])
print('  Times within suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) )

suffixes_present = dfc['custom_suffix'].unique()
print(suffixes_present)

In [ ]:
dfc_uncleaned = dfc

In [ ]:
suffs = [f'CNslide_trim0_dhittw1_s_scX0Y2c0_drall_dtall_{freq}_sh0.232' for freq in ['broad','beta','gamma']]
suffs

In [ ]:
dfc = dfc_uncleaned.query('custom_suffix in @suffs')

In [ ]:
suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall_broad_sh0.232'
if suff not in suffixes_present:
    suff = suffixes_present[0]
tmins_good = dfc.query('custom_suffix == @suff')['tmin'].unique()
std = np.diff( tmins_good.astype(float) ).std() 
print(std)
assert std < 1e-2 # recal we round to two digits
dfc_clean2 = dfc.query('tmin in @tmins_good')
display( dfc_clean2.groupby('custom_suffix')['mtime'].describe(datetime_is_numeric=True) )

In [ ]:
#dfc_ = dfc.query('dec_type == "classic"')

In [ ]:
#dfc = dfc.query('custom_suffix == @suff')

In [ ]:
dfc = dfc_clean2

### multi calcs mgmt

In [ ]:

#reset_index()['tmin'].apply()

In [ ]:
sz.reset_index()[0].unique()

In [ ]:
# all
grp['mtime'].describe(datetime_is_numeric=True)['max']

In [ ]:
dfc_allcollected['mtime'].dt?

In [ ]:
dfc_allcollected['mtime_sec'] = dfc_allcollected['mtime'].values.astype(np.int64) // 10 ** 9

In [ ]:
cols_unique_

In [ ]:
cols_unique_

In [ ]:
dfc_allcollected_.reset_index(drop=True)

In [ ]:
dfc_allcollected_ = dfc_allcollected.sort_values('mtime_sec').reset_index(drop=True)
grp = dfc_allcollected_.groupby(cols_unique_)
sz =  grp.size()
print('Within suffix: min dup = {}, max dup = {}'.\
            format(sz.min(), sz.max() ) )
#grp = dfc_allcollected_.groupby(cols_unique_2)
grp = dfc_allcollected_.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True)['max'] )

In [ ]:
#dfc_allcollected_.groupby(cols_unique_2).size()

In [ ]:
grp = dfc_allcollected_.groupby(cols_unique_)
idx = grp['mtime'].idxmax()

In [ ]:
len(idx), len(dfc_allcollected_)

In [ ]:
grp['mtime'].max?

In [ ]:
grp.max('mtime').reset_index()#.query('custom_suffix == "CNslide_trim0_dhittw0_s_scX1Y1_drall_dtall"')

In [ ]:
grp.max('mtime').reset_index().sort_values('mtime')['mtime']

In [ ]:
dfc_allcollected_.loc[idx].sort_values('mtime')['mtime']

In [ ]:
# select max time within suffix
dfc_clean = aggRows(dfc_allcollected_, 'mtime_sec', 
                   operation= 'max', coltake='corresp', colgrp = cols_unique_)
grp = dfc_clean.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) ['max'] )
#dfc = dfc_clean

In [ ]:
dfc_clean = dfc_allcollected_.sort_values('mtime', ascending=False).drop_duplicates(cols_unique_)
grp = dfc_clean.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) ['max'] )

In [ ]:
dftmp0 = dfc_allcollected_.sample(1000).sort_values('mtime')

In [ ]:
dftmp1 = dftmp 

In [ ]:
dftmp2 = dftmp 

In [ ]:
dftmp3 = dftmp 

In [ ]:
# select max time within suffix
difsec = 0
while difsec < 1e-10:
    dftmp = dftmp3.sample(5).sort_values('mtime')
    #dftmp = dftmp1.sample(20).sort_values('mtime')
    #dftmp = dftmp0.sample(100).sort_values('mtime')
    #dftmp = dftmp0
    grp = dftmp.groupby('custom_suffix')
    desc_max0 = grp['mtime'].describe(datetime_is_numeric=True) ['max']
    display(desc_max0  )
    dfc_clean = aggRows(dftmp, 'mtime_sec', 
                       operation= 'max', coltake='corresp', colgrp = cols_unique_)
    grp = dfc_clean.groupby('custom_suffix')
    desc_max = grp['mtime'].describe(datetime_is_numeric=True) ['max']
    display(desc_max  )
    #dfc = dfc_clean
    difsec = (desc_max0.iloc[0] - desc_max.iloc[0]).total_seconds()
    print(difsec)

In [ ]:
grp0 = dftmp.groupby(cols_unique_)
grp0['mtime'].idxmax()

In [ ]:
cols_unique_

In [ ]:
grp0['mtime'].max().reset_index()

In [ ]:
set(cols_unique) ^ set(cols_unique_)

In [ ]:
cols = ['subject',
 'env',
 'tmin',
 'dec_type',
 'custom_suffix', 'mtime']

In [ ]:
dftmp[cols].sort_values('mtime')

In [ ]:
# remove 4164 -- compared to 5172 same everything except custom suffix
dfc_clean[cols].sort_values('mtime')

In [ ]:
dfc = dfc_allcollected

In [ ]:
from behav_proc import aggRowsws

In [ ]:
from datetime import timedelta

In [ ]:
tdm.total_seconds()

In [ ]:
tdm = dfc['mtime'].diff().min()

In [ ]:
isinstance( date_cutoff, datetime, )

In [ ]:
len(dfc_clean), len(dfc)

In [ ]:
#dfc.groupby(cols_unique_)

In [ ]:
date_cutoff = parser.parse('02 june 2023 15:00')
#date_cutoff = parser.parse('02 june 2023 15:00')
dfc_cut = dfc[dfc['mtime'] > date_cutoff]

In [ ]:
dfc['time_locked'].unique()

In [ ]:
from postproc import extractB2Bcols

In [ ]:
dfc = dfc.query('custom_suffix.str.startswith("CNslide")'
                ' and control_type == "movement" and time_locked == "target"')
dfc_, df_b2b_plot = extractB2Bcols(dfc)

sz =  dfc.groupby(cols_unique_).size()
print('min dup = {}, max dup = {}'.\
            format(sz.min(), sz.max() ) )

In [ ]:
#hours_since

In [ ]:
np.max(days_since)

In [ ]:
grp.groups.keys()

### hist of times

In [ ]:
sns.set(font_scale=1.)
import matplotlib.dates as mdates
from datetime import datetime, timedelta
grp = dfc.groupby(['custom_suffix'] )
for suff,inds in grp.groups.items():
    df_ = dfc.loc[inds]
    date_list = list( sorted( df_['mtime'] ) )
    #date_st = datetime(2023,5,31)
    date_st = datetime(2023,6,2)
    # Calculate the number of days since May 31st and the number of minutes since the corresponding day start
    days_since = [(date - date_st).days for date in date_list]
    hours_since = [(date - date_st).components.hours for date in date_list]
    minutes_since_ds = [(date - date.replace(hour=0, minute=0)).total_seconds() / 60 for date in date_list]
    minutes_since = [(date - date_st).total_seconds() / 60 for date in date_list]

    # Create a histogram of the dates
    fig = plt.figure(figsize=(10,3))
    ax = plt.gca()
    ax.hist(minutes_since, bins=30)

    # Set the x-axis tick format
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    dts = [date_st + timedelta(days=int(day), minutes=int(minute)) for day,minute in zip(days_since, minutes_since)]
    xts = [f'{int(day)}\n{int(minute)}\n{date.strftime("%H:%M")}' for day,minute,date in zip(days_since, minutes_since_ds,dts)]
    #ax.set_xticks(range(len(days_since)))
    
    def ff(x,f):
        x = int(x)
        #print(x,f)
        r = str(x)
        if x < len(days_since):
            r = f'{int(days_since[(x)])}\n{int(minutes_since_ds[(x)])}\n{date_list[(x)].strftime("%H:%M")}'
        return r
    ax.xaxis.set_major_formatter(ff)
    
    #ax.set_xticklabels(xts)

    # Set the x-axis label and title
    ax.set_xlabel(f'Days Since {date_st} and Minutes Since Day Start')
    ax.set_ylabel('Frequency')
    ax.set_title('Histogram of Dates ' + suff)
    
    #break

    #plt.show()

In [ ]:
len(days_since)

In [ ]:
tmins_all = dfc.tmin.unique()
print(tmins_all), len(tmins_all)

In [ ]:
#tmins_to_plot = ['-0.36', '-0.13', '0.10', '0.34']
tmins_to_plot = ['-1.06', '-0.82', '-0.59', '-0.36', '-0.13', '0.10', '0.34', '0.57', '0.80', '1.03']
tmins_to_plot = tmins_all

In [ ]:
#dfc.columns

#### reshape debug

In [ ]:
display(arr_reshaped)

In [ ]:
arr.reshape(8, 3) - arr.transpose(0,2,1).reshape(8, 3)

In [ ]:
arr_reshaped.shape, arr.transpose(0,2,1).shape

In [ ]:
arr_back[:,0,:], arr_back[:,1,:]

In [ ]:
arr_back[0,:,:],arr_back[1,:,:]

In [ ]:
import numpy as np

arr = np.arange(24).reshape(2, 3, 4)
arr[:,1,:] *= 10
arr[:,2,:] *= 100
print("Original array:\n", arr)

# Reshape to 3x8 array
arr_reshaped = arr.transpose(0,2,1).reshape(-1, 3)
print("\nReshaped array:\n", arr_reshaped)
from sklearn.preprocessing import RobustScaler as robsc
arr_reshaped1 = robsc(with_centering=True, unit_variance=False).fit_transform(arr_reshaped)

print(arr_reshaped1)

# Reshape back to 2x3x4 array
arr_back = arr_reshaped1.reshape(arr.transpose(0,2,1).shape).transpose((0,2,1))# * 100
print("\nBack to original shape:\n", arr_back)


In [ ]:
tmins_to_plot

In [ ]:
suffixes_present

## continue

In [ ]:
from postproc import extractClassicCols
varnames0 = ['prev_movement','prev_belief','movement','belief','feedback',
             'error','prev_error','err_sens','next_error','correction' ]
df_class_plot = extractClassicCols(dfc, defvarcolns=varnames0,
                                  error_handling = 'ignore')

In [ ]:
from postproc import extractB2Bcols
try:
    dfc_, df_b2b_plot0 = extractB2Bcols(dfc, toclean=varnames0)
except Exception as e:                    
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]    
    #display(ei.tb_frame)
    psf = ei

    lfprev = None; lf = None
    lfs = []
    di = 0
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame        

        psf = psf.tb_next  
        if ('conda' not in stackframe.f_code.co_filename) and\
            ('mamba' not in stackframe.f_code.co_filename):        
            display(di, stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            di += 1
        if psf is None:
            break    

    #raise e
    print('EXC',e)

In [ ]:
df_b2b_plot = pd.concat([df_class_plot,df_b2b_plot0] )
def f(row):
    x = row['vals_to_plot']
    from postproc import ListWrap
    if isinstance(x, ListWrap):
        x = x.data    
    else:
        if not isinstance(x,float):
            print(type(x))
    if isinstance(x, list) or isinstance(x, np.ndarray):        
        if len(x ) != 1:
            print(row['dec_type'],row)
            raise ValueError('ee')
        x = x[0]
    return x
df_b2b_plot['vals_to_plot'] = df_b2b_plot.apply(f,1)

rnd = df_b2b_plot.query('env == "random"').set_index(['subject'] + cols0)
stb = df_b2b_plot.query('env == "stable"').set_index(['subject'] + cols0)

dfconddif = stb['vals_to_plot'] - rnd['vals_to_plot']
dfconddif = dfconddif.to_frame().rename(columns={'vals_to_plot':'difcond'})

In [ ]:
#get tmins
tl2tmins_to_plot = {}
tl2indhome = {}
for tl in df_b2b_plot['time_locked'].unique():    
    tmins_to_plot  = list(sorted( df_b2b_plot.query('time_locked == @tl')['tmin'].unique(), key=lambda x: float(x) ) )
    tl2tmins_to_plot[tl] = tmins_to_plot
    print(tl, len(tmins_to_plot))
    
    if '-0.50' in tmins_to_plot:
        indhome = tmins_to_plot.index('-0.50'); print(indhome)
    elif '-0.59' in tmins_to_plot:
        indhome = tmins_to_plot.index('-0.59'); print(indhome)
    else:
        try:
            indhome = tmins_to_plot.index('0.09'); print(indhome)
        except ValueError as e:
            indhome = None
    
    tl2indhome[tl] = indhome
print( tl2indhome )

control_type_present = df_b2b_plot['control_type'].unique(); 
print(control_type_present)
time_lockeds_present = df_b2b_plot['time_locked'].unique(); control_type_present
print(time_lockeds_present)

In [ ]:
df_b2b_plot.groupby(['dec_type','varname']).size()

In [ ]:
#dfconddif

In [ ]:
# cols_ = list( set(cols_unique) - set(['env']) ); cols_
# # takes several minutes
# grp = df_b2b_plot.groupby(cols_ + ['varseti','varname'])
# assert grp.size().max() == 2

# def f(dftmp):
#     assert set( dftmp['env'].unique() ) == set(['random','stable'])
#     d = dftmp.set_index('env').to_dict('index')
#     r = d['stable']['vals_to_plot'] - d['random']['vals_to_plot']
#     return r
# dfconddif = grp.apply(f)
# dfconddif= dfconddif.reset_index(name='difcond')

In [ ]:
#display(grp.size().describe())

In [ ]:
cols0 = ['custom_suffix','time_locked','control_type','dec_type','varseti','varname','tmin']
cols_ = list( set(cols_unique) - set(['env']) ); cols_

In [ ]:
# compare mean vs random decoding
from base2 import decod_stats # X - participant x times
def f(dftmp):
    r = dftmp.pivot(index='subject', columns='tmin', values='difcond')
    time_order = r.columns.tolist()
    #print(r.shape)  #participant x times
    r = r.to_numpy()
    pvals =  decod_stats(r, n_jobs=4, n_permutations = 2**10)
    res = pd.DataFrame({'pvalue': pvals, 'tmin':time_order})#.set_index('tmin')
    #display(res)
    return res
cols0 = ['custom_suffix','time_locked','control_type','dec_type','varseti','varname','tmin']
grp = dfconddif.groupby(cols0)
assert grp.size().max() == 20,  grp.size().max()
dfconddif_pv = dfconddif.reset_index().groupby(list(set(cols0) - set(['tmin']))).apply(f)
dfconddif_pv['signif'] = dfconddif_pv['pvalue'] < 0.05

In [ ]:
# compare stable dec with zero
from base2 import decod_stats # X - participant x times
def f(dftmp):
    r = dftmp.pivot(index='subject', columns='tmin', values='vals_to_plot')
    time_order = r.columns.tolist()
    #print(r.shape)  #participant x times
    r = r.to_numpy()
    pvals =  decod_stats(r, n_jobs=4, n_permutations = 2**10)
    res = pd.DataFrame({'pvalue': pvals, 'tmin':time_order})#.set_index('tmin')
    #display(res)
    return res

cols_ = list( set(cols_unique) - set(['subject'])); #cols_
cols__ = cols_ + ['varseti','varname']
print(cols__)
df_ = df_b2b_plot.query('env == "stable"')
grp = df_.groupby(cols__)
#grp = dfconddif.groupby(['custom_suffix','varseti','varname','control_type','tmin'])
assert grp.size().max() == 20, grp.size().max()
dfconddif0_pv = df_.groupby(list(set(cols__) - set(['tmin']))).apply(f)
dfconddif0_pv['signif'] = dfconddif0_pv['pvalue'] < 0.05

In [ ]:
#cols_unique

In [ ]:
gc.collect()

In [ ]:
#suffixes_present

In [ ]:
#df_b2b_plot.columns

In [ ]:
cols0

In [ ]:
#df_b2b_plot.groupby(['subject'] + cols0 + ['env']).size()

In [ ]:
#df_b2b_plot.groupby(cols0).size()

In [ ]:
df_b2b_plot.groupby(cols0)['subject'].nunique()

In [ ]:
gc.collect()

In [ ]:
from postproc import runStatTests
df_ttest_res = runStatTests(df_b2b_plot, cols0, 'two-sided', 
                            verbose=0)

In [ ]:
verbose = 0
# takes a long time to finish
from scipy.stats import ttest_rel, ttest_1samp
alt = 'two-sided'
def f(df):
    vsd = {}
    vsl = []
    resd = {'stable_greater_random':np.nan}
    
    # choosing subject present for both env
    subjs = []
    for env_cur in ['stable', 'random']:        
        ss = df.query('env == @env_cur')['subject'].unique()
        subjs += [ss]
    subj_both = list(set(subjs[0]) & set(subjs[1]))
    
    for env_cur in ['stable', 'random']:        
        dftmp = df.query('env == @env_cur')            
        if len(dftmp) == 0:
            #print(f'zero len for {ct}: {varname}, skipping')
            continue
        assert len(dftmp) <= 20, len(dftmp)
        vsd[env_cur] = dftmp['vals_to_plot'].values
        
        dftmp = dftmp.query('subject in @subj_both')        
        vsl += [dftmp['vals_to_plot'].values]
    if len(vsd) == 0:
        return None
    for env_cur in ['stable', 'random']:
        if env_cur not in vsd:
            print(f'No {env_cur} found')
            print(df[cols0].iloc[0])
            resd = pd.DataFrame([resd])
            return resd
                
        tr = ttest_1samp( vsd[env_cur], 0, alternative=alt )
        s = ' '
        if tr.pvalue < 0.05:
            s = '*'
        #if (tr.pvalue < 0.05) or (not signif_only):
        #    print(s + '{} control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format(tmin, ct, varname, env_cur, tr.pvalue) )
        resd[env_cur] = tr.pvalue
        resd[env_cur + '_len'] = len(vsd[env_cur])
    if len(vsl[0]) != len(vsl[1]):
        print('Not equal!', (len(vsl[0]), len(vsl[1])) )
        print(len(subj_both))
        print(df[cols0].iloc[0])
        resd = pd.DataFrame([resd])
        return resd
    
    tr = ttest_rel(vsl[0],vsl[1], alternative=alt )
    
    s = ' '
    if tr.pvalue < 0.05:
        s = '*'
        #tmins_good += [tmin]
    #if (tr.pvalue < 0.05) or (not signif_only):
    #    print(s + '{} control={:8}, var={:14}, stable>random pvalue={:.4f}'.format(tmin, ct, varname, tr.pvalue) )
    #print('')
    if alt == 'greater':
        resd['stable_greater_random'] = tr.pvalue
    else:
        resd['stable_neq_random'] = tr.pvalue
    #resd['tmin'] = tmin
    #resd['varname'] = varname
    #resd['control_type'] = ct
    #resd['custom_suffix'] = suff
    #resd['varseti'] = varseti
    resd = pd.DataFrame([resd])
    #rows += [resd]
    return resd
df_ttest_res = df_b2b_plot.groupby(cols0).apply(f)
  

In [ ]:
df_b2b_plot.query('dec_type == "classic"').iloc[0]

In [ ]:
#cols0

In [ ]:
#df_ttest_res

In [ ]:
df_ttest_res_basic = df_ttest_res.query('varseti == 0')
df_ttest_res_ES = df_ttest_res.query('varseti == 1')

display( df_ttest_res_ES['stable_len'].describe(),
        df_ttest_res_basic['stable_len'].describe() )

#df_ttest_res_sig = df_ttest_res.query('stable_greater_random <= 0.05')
df_ttest_res_sig = df_ttest_res.query('stable_neq_random <= 0.05')
print(len(df_ttest_res_sig))

In [ ]:
#df_ttest_res.index._names, df_ttest_res.columns

In [ ]:
# Do T-tests (old)
#suff = 'trim0_dhittw1_s_scX0Y0_drall_dtall'
#suff = 'CNhome_trim0_dhittw1_s_scX1Y1_drall_dtall'
signif_only = 1
rows = []
for varseti in [0,1]:
    qs = ('varseti == @varseti and custom_suffix == @suff')
    for suff in suffixes_present:
        print('----------- ' ,suff)    
        for ct in control_type_present: #, 'feedback']:
            for varname in ['err_sens', 'correction', 
                    'prev_error', 'error', 'prev_belief', 'prev_movement', 'prev_feedback']:
                tmins_good = []
                for tmin in tmins_to_plot:
                    vsd = {}
                    vsl = []
                    resd = {}
                    for env_cur in ['stable', 'random']:
                        qs2 = qs + ' and env == @env_cur and control_type == @ct and varname == @varname'
                        qs2 += ' and tmin == @tmin'
                        dftmp = df_b2b_plot.query(qs2)            
                        if len(dftmp) == 0:
                            #print(f'zero len for {ct}: {varname}, skipping')
                            continue
                        assert len(dftmp) <= 20, len(dftmp)
                        vsd[env_cur] = dftmp['vals_to_plot'].values
                        vsl += [dftmp['vals_to_plot'].values]
                    if len(vsd) == 0:
                        continue
                    for env_cur in ['stable', 'random']:
                        tr = ttest_1samp( vsd[env_cur], 0, alternative='greater' )
                        s = ' '
                        if tr.pvalue < 0.05:
                            s = '*'
                        if (tr.pvalue < 0.05) or (not signif_only):
                            print(s + '{} control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format(tmin, ct, varname, env_cur, tr.pvalue) )
                        resd[env_cur] = tr.pvalue
                        resd[env_cur + '_len'] = len(vsd[env_cur])
                    tr = ttest_rel(vsl[0],vsl[1], alternative='greater')
                    s = ' '
                    if tr.pvalue < 0.05:
                        s = '*'
                        tmins_good += [tmin]
                    if (tr.pvalue < 0.05) or (not signif_only):
                        print(s + '{} control={:8}, var={:14}, stable>random pvalue={:.4f}'.format(tmin, ct, varname, tr.pvalue) )
                    print('')
                    resd['stable_greater_random'] = tr.pvalue
                    resd['tmin'] = tmin
                    resd['varname'] = varname
                    resd['control_type'] = ct
                    resd['custom_suffix'] = suff
                    resd['varseti'] = varseti
                    rows += [resd]
            
            

df_ttest_res = pd.DataFrame(rows)


In [ ]:
cols0

In [ ]:
# just to see numbers, not used later
df_ttest_res_ES_sig = df_ttest_res_ES.query('stable_neq_random <= 0.05').reset_index()
display( df_ttest_res_ES_sig.groupby(cols0[:-1]).size().max() )
df_ttest_res_basic_sig = df_ttest_res_basic.query('stable_neq_random < 0.05').reset_index()
#df_ttest_res_basic_sig.groupby(['time_locked','custom_suffix','control_type','varname']).size()
display( df_ttest_res_basic_sig.groupby(cols0[:-1]).size().max() )

In [ ]:
#qs = 'stable_greater_random <= 0.05 and stable <= 0.05'
qs = 'stable_neq_random <= 0.05 and stable <= 0.05'
df_ttest_res_ES_sig2 = df_ttest_res_ES.query(qs)
df_ttest_res_basic_sig2 = df_ttest_res_basic.query(qs)

display( df_ttest_res_ES_sig2.groupby(cols0[:-1]).size() )
display( df_ttest_res_basic_sig2.groupby(cols0[:-1]).size() )

### dbg

In [ ]:
#set(tmins_good) ^ set(tmins_to_plot), len((tmins_to_plot)), len(tmins_good)

In [ ]:
#
#dfconddif_pv.sort_values?
#dfconddif_pv
#dfconddif_pv.loc[(0, varname)]#.query('signif == True')#['tmin']
#dfconddif_pv
#dfconddif0_pv

In [ ]:
df_ttest_res_sig.index._names

In [ ]:
cols0[:-1]

In [ ]:
# indcols = df_ttest_res_sig.index._names[:-1]
# for df_ in dfconddif0_pv, dfconddif_pv, df_ttest_res_sig:
#     df_.reset_index(drop=False, inplace=True)
#     df_.set_index(indcols, inplace=True)

In [ ]:
#tpl = (tl, ct, 'stable',slice(None), suff, varseti,'b2b', varname)

In [ ]:
#suffixes_present

In [ ]:
#df_ttest_res_sig.index._names

In [ ]:
#df_ttest_res_sig = df_ttest_res_sig.reset_index()

In [ ]:
#dec_types_found

In [ ]:
#dfsub = dfconddif_pv.loc[tpl].reset_index(drop=False)

In [ ]:
tpl

In [ ]:
df_ttest_res_sig.loc[tpl]

In [ ]:
gc.collect()

In [ ]:
dec_types_found

In [ ]:
df_b2b_plot.groupby(['subject','custom_suffix','dec_type']).size()

In [ ]:
df_b2b_plot.groupby('dec_type').size()

In [ ]:
dec_type = 'classic'

In [ ]:
suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall_broad_sh0.232'

In [ ]:
suff,tl,dec_type,varseti

In [ ]:
df_ = df_b2b_plot_.query(qs + ' and varseti == @varseti')
len(df_)

## plot

In [ ]:
del df_b2b_plot_
gc.collect()

In [ ]:
qs_ = qs + ' and varseti == 0'
df_ = df_b2b_plot_.query(qs_)
df_g = df_[~df_['vals_to_plot'].isna()]
df_b = df_[df_['vals_to_plot'].isna()]
print(len(df_g),len(df_b))

In [ ]:
qs_ = qs + ' and varseti == -1'
df_ = df_b2b_plot_.query(qs_)
df_g = df_[~df_['vals_to_plot'].isna()]
df_b = df_[df_['vals_to_plot'].isna()]
print(len(df_g),len(df_b))

In [ ]:
sns.relplot(data=df_g, **pargs)

In [ ]:
assert 

In [ ]:
# main plot
import re
df_b2b_plot_ = df_b2b_plot.copy()
df_b2b_plot_['tmin'] = df_b2b_plot_['tmin'].astype(float)

In [ ]:
df_['dec_type']

In [ ]:
time_locked = 'target'; dec_type='b2b'; varseti=-1
df_b2b_plot_.query(qs_)['varname'].unique()

In [ ]:
#dfconddif_pv['tmin'] = dfconddif_pv['tmin'].astype(float)
#dfconddif_pv= dfconddif_pv.sort_values('tmin')

dfconddif0_pv = dfconddif0_pv.reset_index().set_index(cols0[:-1])
dfconddif_pv = dfconddif_pv.reset_index().set_index(cols0[:-1])
df_ttest_res_sig= df_ttest_res_sig.reset_index().set_index(cols0[:-1])
df_ttest_res_basic_sig2 = df_ttest_res_basic_sig2.reset_index().set_index(cols0[:-1])


sns.set(font_scale=1.5)
import seaborn as sns
#suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall'
# 'CNslide_trim0_dhittw0_s_scX1Y1_drall_dtall'

pargs = dict(kind='line',  col='control_type',  row='varname',
           x = 'tmin', y = 'vals_to_plot',   hue='env', 
             hue_order = ['stable','random'],
             palette=['orange', 'grey'], aspect=3, 
             facet_kws={'sharey': False} )

for suff in  suffixes_present:
    for tl in time_lockeds_present:  
        for dec_type in dec_types_found:
            #qs = 'control_type == @ct and custom_suffix == @suff'
            qs = ('time_locked == @tl and custom_suffix == @suff '
                'and dec_type == @dec_type')
            suff_short = suff[14:].replace('_drall_dtall','').\
                replace('_sh0.232','').replace('_s','')

            #tick_skip = 2
            fignames = ['classic','basic_vars','ES']
            for varseti, fign in zip([-1, 0,1],fignames ):
                qs_ = qs + ' and varseti == @varseti'
                df_ = df_b2b_plot_.query(qs_)
                df_g = df_[~df_['vals_to_plot'].isna()]
                if not len(df_):
                    continue
                try:
                    fg = sns.relplot(data=df_g, **pargs)
                except KeyError as e:
                    print('Err during relplot ',suff,tl,
                          dec_type,varseti,e)
                    plt.close()
                    continue
                axsf = fg.axes.flatten()
                for axi,ax in enumerate(axsf):
                    ax.axhline(y=0, c='r', ls=':', lw=3)
                    ax.axvline(x=-0.5, c='r', ls=':', lw=3)
                    ttl = ax.title.get_text()
                    mr = re.match('varname = (\w*) \| control_type = (\w*)', ttl)
                    if mr is None:
                        continue
                    mrg = mr.groups()
                    varname = mrg[0]
                    ct = mrg[1]

                    ymin,ymax = ax.get_ylim()

                    tpl = (suff,tl,ct,dec_type,varseti, varname)
                    # * markers are individ ttest > 0 and stable > random
                    try:
                        tmins_good = df_ttest_res_sig.loc[tpl]  
                        tmins_good = tmins_good.reset_index(drop=False)['tmin'].astype(float)
                        ax.plot( tmins_good, 
                                len(tmins_good) * [ymax * 1.1], lw=0, 
                                marker='*', label='stable > random naive')
                    except KeyError as e:
                        tmins_good = []
                        
                    # x markers are individ ttest > 0 and stable > random
                    try:
                        tmins_stable_nonz = dfconddif0_pv.loc[tpl].query('signif == True').reset_index(drop=False)
                        tmins_stable_nonz = tmins_stable_nonz['tmin']
                        ax.plot( tmins_stable_nonz.astype(float), 
                            len(tmins_stable_nonz) * [ymax * 1.3], 
                            lw=0, marker='x', label='stable > 0')    
                    except KeyError as e  :
                        tmins_stable_nonz = []

                    try:
                        dfsub = dfconddif_pv.loc[tpl].reset_index(drop=False)
                        dfsub['tmin'] = dfsub['tmin'].astype(float)
                        dfsub = dfsub.sort_values('tmin')
                        tmins_stable_g_random,sig = dfsub[['tmin','signif']].to_numpy().T                    
                        # plus markers are same as shading    
                        sig = sig.astype(bool)
                        tmins_stable_g_random = tmins_stable_g_random.astype(float)
                        ax.fill_between(tmins_stable_g_random, 
                            ymax, where = sig, color='red', alpha=0.1)                        
                        ax.plot( tmins_stable_g_random[sig], 
                            np.sum(sig) * [ymax * 1.2], 
                            lw=0, marker='+', label='stable != random')
                    except KeyError as e:
                        tmins_stable_g_random,sig = [],[]
                    print(varname, len(tmins_good), len(tmins_stable_g_random), len(tmins_stable_nonz))

                    #xticks = ax.get_xticks()[::tick_skip]
                    #ax.set_xticks(xticks)
                    ax.grid(True)
                    if axi == 3:
                        ax.legend(loc='lower right')
                try:
                    sns.move_legend(fg, loc='upper right')
                    plt.suptitle(f'{dec_type} time locked = {tl}')
                    plt.savefig( pjoin(path_fig, f'{dec_type}_slide_relplot_{suff_short}_{tl}_{fign}.pdf'))                    
                    plt.close()
                except ValueError as e:
                    print(e)
                print(f'{dec_type} varseti={varseti} fig done')
            #break
        #break
    #break

In [ ]:
dur = 0.464 
ds = np.arange( -dur*.5 * 64, dur*.5 * 65 , dur/2 ) - 0.5 + dur


In [ ]:
ds[0], ds[-1]

In [ ]:
ds[60:-60] 

In [ ]:
print(ttl)
mrg = re.match('varname = (\w*) | control_type = (\w*)', ttl).groups()
mrg

In [ ]:
0.464 / 4

## Many boxplots

In [ ]:
sns.set(font_scale=1.5)

import seaborn as sns
#suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall'
suff = 'CNslide_trim0_dhittw0_s_scX1Y1_drall_dtall'
qs = 'tmin.isin(@tmins_to_plot) and control_type == "movement" and custom_suffix == @suff'
pargs = dict(kind='boxen',  col='custom_suffix',  row='varname',
           x = 'tmin', y = 'vals_to_plot',   hue='env', 
             palette=['orange', 'grey'], aspect=9,
            sharey = None, sharex = None)

tick_skip = 3

fg = sns.catplot(data=df_b2b_plot.query(qs + ' and varseti == 0'), **pargs)
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':', lw=3)
    ax.axvline(x=indhome, c='r', ls=':', lw=3)
    ax.grid()
    varname = re.match('varname = (\w*) |.*', ax.title.get_text() ).groups()[0]
    tmins_good = df_ttest_res_basic_sig.query('varname == @varname')['tmin'].values    
    tmins_good2 = dfconddif_pv.loc[(suff, 0, varname)].query('signif == True')['tmin']
    
    ymin,ymax = ax.get_ylim()
    ax.plot( tmins_good, len(tmins_good) * [ymax * 1.1], lw=0, marker='*')
    ax.plot( tmins_good2, len(tmins_good2) * [ymax * 1.3], lw=0, marker='+')
    print(varname, len(tmins_good2))
    
    xticks = ax.get_xticks()[::tick_skip]
    ax.set_xticks(xticks)
sns.move_legend(fg, loc='upper right')
plt.tight_layout()
plt.savefig( pjoin(path_fig, 'b2b_slide_basic_vars.pdf'))
print('1st fig done')

fg = sns.catplot(data=df_b2b_plot.query(qs + ' and varseti == 1'), **pargs)
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':', lw=3)
    ax.axvline(x=indhome, c='r', ls=':', lw=3)
    ax.grid()
    varname = re.match('varname = (\w*) |.*', ax.title.get_text() ).groups()[0]
    tmins_good= df_ttest_res_ES_sig.query('varname == @varname')['tmin'].values
    tmins_good2 = dfconddif_pv.loc[(suff, 1, varname)].query('signif == True')['tmin']
    ymin,ymax = ax.get_ylim()
    ax.plot( tmins_good, len(tmins_good) * [ymax * 1.1], lw=0, marker='*')
    ax.plot( tmins_good2, len(tmins_good2) * [ymax * 1.3], lw=0, marker='+')
    print(varname, len(tmins_good2))
    
    xticks = ax.get_xticks()[::tick_skip]
    ax.set_xticks(xticks)


sns.move_legend(fg, loc='upper right')
plt.tight_layout()
plt.savefig( pjoin(path_fig, 'b2b_slide_ES.pdf'))

In [ ]:
ax.get_xticks()

## Plot home from slide

In [ ]:
sns.set(font_scale=1.1)
import seaborn as sns
pargs = dict(kind='box',  col='control_type',  row='varseti',
               x = 'varname', y = 'vals_to_plot',   hue='env', palette=['orange', 'grey'] )

for suff in df_b2b_plot['custom_suffix'].unique():
    suff_short = suff[14:].replace('_drall_dtall','').\
        replace('_sh0.232','').replace('_s','') 
    
    fg = sns.catplot(data=df_b2b_plot.query(
        'varseti == 0 and custom_suffix == @suff and tmin == "-0.50"'), **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.grid()

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()    
    plt.savefig( pjoin(path_fig, f'b2b_home_basic_vars_{suff_short}.pdf'))

    df_ = df_b2b_plot.query('varseti == 1 and custom_suffix == @suff and tmin == "-0.50"')
    fg = sns.catplot(data=df_, **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()

    plt.savefig( pjoin(path_fig, f'b2b_home_ES_{suff_short}.pdf'))

# Plot home

In [ ]:
#for subject in subjects[:1]:
#%debug
import gc; gc.collect()
plot_diff_vs_es = 0
plot_diff = 0

regression_type = 'Ridge'
time_lockeds=['target', 'feedback']
#control_type = 'movement'
#subject_inds = [0]
subject_inds = np.arange(len(subjects))
require_all_subjects = False
#subject_inds = list(range(12)) + list(range(13,20))
#subject_inds = [12]

# scale robust = 1
#time_start = parser.parse('14:40 of 31 may 2023'); 
#time_end = parser.parse('21:39 of 31 may 2023'); 

# scale robust = 0
#time_start = parser.parse('13:00 of 14 june 2023'); 
time_start = parser.parse('17:30 of 27 june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 

custom_prefix = None

use_preload_df = 0
load_df        = 0
exit_after = 'collect'
plot_scores_per_subj = 0
ktes = ['err_sens','correction','error', 'prev_error' ]
control_types = ['movement', 'feedback']
varnames = ktes
#tmins = ['-5.00']
tmin_desired = '-0.50'
#tmin_desired = None

import traceback
DEBUG = 1

save_df = 0
#%debug

failed = []
dfs = []
for hpass in ['no_filter']: #,'0.1']:
    #for env in ['stable','random']:
    for env in [['stable','random']]:
        #for freq_name in list(freq_name2freq.keys() ):
        for freq_name in ['broad']: #, 'beta']:
            for time_locked in time_lockeds:
                for control_type in control_types:

                    s=  ','.join( getAnalysisVarnames(time_locked, control_type)[0] )
                    varnames_b2b = [s, 'err_sens,prev_error,error']

                    output_folder = f'corr_spoc_es_sliding2_{hpass}'        
                    script_name = pjoin(scripts_dir,
                            'correlation_error_sensitivity_spoc_decoding_sliding.py')
                    try:
                        ipy.magic('%run -i {script_name}')                  
                    except Exception as e:                    
                        print('EXC',e)
                        exc_info = sys.exc_info()
                        exc = traceback.TracebackException(*exc_info, capture_locals=True)
                        ei = exc_info[2]    
                        #display(ei.tb_frame)
                        psf = ei

                        lfprev = None; lf = None
                        lfs = []
                        di = 0
                        while psf.tb_frame is not None:
                            stackframe = psf.tb_frame
                            display(di, stackframe)

                            psf = psf.tb_next  
                            if 'conda' not in stackframe.f_code.co_filename:        
                                lfprev = lf
                                lf = stackframe.f_locals
                                lfs += [lf]
                                di += 1
                            if psf is None:
                                break    

                        raise e

                        print('Exc for ', hpass,freq_name)
                        failed += [ (hpass, freq_name)]
                        print('EXC',e)
                    #df.to_pickle( pjoin(path_fig, save_folder, 'df') )
                    dfs += [df]
                if not DEBUG:
                    del df
                import gc; gc.collect()
                plt.close()
            #break
        #break
    #break

dfc =pd.concat(dfs)

In [ ]:
dfc['custom_suffix'].unique()

In [ ]:
set( df['subject'] ) ^ set(np.array(subjects)[subject_inds])

In [ ]:
dfc['mtime'].min(), dfc['mtime'].max()

In [ ]:
cols_unique = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'tmin', 'tmax', 'dec_type',
              'custom_suffix']
if dfc.duplicated(cols_unique).any():
    display( dfc[ dfc.duplicated(cols_unique) ] )
else:
    print('No duplicates!')

print( len(dfc), dfc['subject'].nunique(), dfc['control_type'].nunique(), subject_inds )

In [ ]:
from postproc import  extractB2Bcols
#%debug
varnames0 = ['prev_movement','prev_belief','movement','belief','feedback',
             'error','prev_error','err_sens','next_error','correction' ]
dfc_, df_b2b_plot = extractB2Bcols(dfc, toclean=varnames0)

suffixes_present = df_b2b_plot['custom_suffix'].unique()

In [ ]:
#path_fit_R = pjoin( os.path.expandvars('$HOME'), 'ownCloud', 'Current' ,'_share_Romain' ); path_fit_R

In [ ]:
#df_b2b_plot['SLURM_job_id']

In [ ]:
dftmp['varname'].unique()

In [ ]:
from scipy.stats import ttest_rel

In [ ]:
#suff = 'trim0_dhittw1_s_scX0Y0_drall_dtall'
#suff = 'CNhome_trim0_dhittw1_s_scX1Y1_drall_dtall'
qs = ('varseti == 0 and '
 ' custom_suffix == @suff and tmin == "-0.50"')
signif_only = 1
for suff in suffixes_present:
    print('----------- ' ,suff)
    for ct in ['movement', 'feedback']:
        for varname in ['prev_movement', 'prev_error', 'error', 'prev_belief',
               'prev_feedback']:
            vsd = {}
            vsl = []
            for env_cur in ['stable', 'random']:
                qs2 = qs + ' and env == @env_cur and control_type == @ct and varname == @varname'
                dftmp = df_b2b_plot.query(qs2)            
                if len(dftmp) == 0:
                    #print(f'zero len for {ct}: {varname}, skipping')
                    continue
                assert len(dftmp) == 20, len(dftmp)
                vsd[env_cur] = dftmp['vals_to_plot'].values
                vsl += [dftmp['vals_to_plot'].values]
            if len(vsd) == 0:
                continue
            for env_cur in ['stable', 'random']:
                tr = ttest_1samp( vsd[env_cur], 0, alternative='greater' )
                s = ' '
                if tr.pvalue < 0.05:
                    s = '*'
                if (tr.pvalue < 0.05) or (not signif_only):
                    print(s + 'control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format( ct, varname, env_cur, tr.pvalue) )
            tr = ttest_rel(vsl[0],vsl[1], alternative='greater')
            s = ' '
            if tr.pvalue < 0.05:
                s = '*'
            if (tr.pvalue < 0.05) or (not signif_only):
                print(s + 'control={:8}, var={:14}, stable>random pvalue={:.4f}'.format( ct, varname, tr.pvalue) )
            print('')
            

In [ ]:
path_fig

In [ ]:
df_b2b_plot['custom_suffix'].unique()

In [ ]:
    suff = 'CNhome_trim0_dhittw0_s_scX1Y1_drall_dtall'
    fg = sns.catplot(data=df_b2b_plot.query(
        'varseti == 1 and custom_suffix == @suff and varname == "err_sens"'), **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()


In [ ]:
catcols = ['control_type', 'env', 'varname', 'time_locked', 'custom_suffix']
for col in catcols:
    df_b2b_plot[col] = df_b2b_plot[col] .astype('category')

In [ ]:
suffixes_desired = ['CNhome_trim0_dhittw1_s_scX0Y0c0_drall_dtall_broad']

In [ ]:
suffixes_present

In [ ]:
sns.set(font_scale=1.1)
import seaborn as sns
for suff in suffixes_desired:
    
    pargs = dict(kind='box',  col='control_type',  
                 #row='varseti',
               x = 'varname', y = 'vals_to_plot',   hue='env',
                 palette=['orange', 'grey'] , sharey='none',
                row='time_locked')
    fg = sns.catplot(data=df_b2b_plot.query(
        'varseti == 0 and custom_suffix == @suff and tmin == "-0.50"'), 
                     **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.grid(True)

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()    
    plt.savefig( pjoin(path_fig, f'b2b_home_basic_vars_{suff}.pdf'))

    fg = sns.catplot(data=df_b2b_plot.query('varseti == 1 and custom_suffix == @suff and tmin == "-0.50"'),
                     **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.grid(True)

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()

    plt.savefig( pjoin(path_fig, f'b2b_home_ES_{suff}.pdf'))

In [ ]:
#ct = "movement"
#env_cur = 'stable'
dfc_cl = dfc.query('dec_type == "classic" and tmin == "-0.50"')
dfc_cl = dfc_cl.copy()
print(len(dfc_cl))
fg = sns.catplot(data=dfc_cl, kind='boxen', x='control_type',
           y = 'prev_error_scores_mean_over_folds', 
                  hue='env')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':')
    
plt.suptitle('prev_error')
plt.savefig( pjoin(path_fig, 'classic_home_prev_error.pdf'))

### debug

In [ ]:
fn = 'stable_trials_Ridge_target_movement_trim0_dhittw1_s_scX0Y0_drall_dtall_broad_t=-0.50,0.00.npz'
regex = r'(stable|random)_(trialwb|trialwe|trialwpert_wb|trialwpert_we|trialwtgt|trialwpert|trialwtgt_we|trialwtgt_wb|trialwtgt_wpert_wb|trialwtgt_wpert_we|trialwpertstage_wb|trialwtgt_wpertstage_wb|trialwtgt_wpertstage_we|trials)_(Ridge|xgboost)_(feedback|target)_(feedback|movement)_(.*)_(broad|theta|alpha|beta|gamma)_t=(.*),(.*)\.npz'
import re
r = re.match(regex,fn)
r.groups()

In [ ]:
#set( df['subject'] ) ^ set(np.array(subjects)[subject_inds])

In [ ]:
fnf = dfc.query('dec_type == "b2b" and control_type == "movement"')['fn_full'].iloc[0]

In [ ]:
f = np.load(fnf, allow_pickle=1)

In [ ]:
fnf

In [ ]:
f['decoding_per_var_b2b'][()]['prev_movement,prev_error,error,prev_belief']['scores']#.keys()

In [ ]:
list(f.keys())

In [ ]:
dfc.query('dec_type == "b2b" and control_type == "movement"')[cols_unique +\
    ['prev_movement,prev_error,error,prev_belief_scores', 'mtime'] ].sort_values('mtime')

In [ ]:
subj = subjects[subject_inds[0]]
dfc.query('subject==@subj and dec_type == "b2b"')[cols_unique +\
    ['prev_feedback,prev_error,error,prev_belief_scores', 'mtime', 'SLURM_job_id'] ].sort_values('mtime')

In [ ]:
dfc['control_type'].unique()

In [ ]:
varnames_b2b

In [ ]:
#dfc_cl['prev_error_scores_mean_over_folds']

In [ ]:
#dfc_cl.columns

In [ ]:
# print(vns)
# dfc_.loc[inds, vns + '_scores']

In [ ]:
#dfc_.loc[inds, vns + '_scores'].apply(pd.Series)

In [ ]:
gc.collect()

In [ ]:
-0.464*4 -0.5,  0.464*8 -0.5

In [ ]:
# varnames_b2b_s = varnames_b2b 
# varnames_b2b = varnames_b2b_s.split(',')
# varnames_b2b

In [ ]:
dfc.columns

In [ ]:
rs =r'(stable|random|all)_(trialwb|trialwe|trialwpert_wb|trialwpert_we|trialwtgt|trialwpert|trialwtgt_we|trialwtgt_wb|trialwtgt_wpert_wb|trialwtgt_wpert_we|trialwpertstage_wb|trialwtgt_wpertstage_wb|trialwtgt_wpertstage_we|trials)_(Ridge|xgboost)_(feedback|target)_(.*)_(broad|theta|alpha|beta|gamma)_t=(.*),(.*)\.npz'
import re
r = re.match(rs, r'random_trials_Ridge_feedback_trim0_dhittw1_s_drall_dtall_broad_t=-0.50,0.00.npz')
r

In [ ]:
r.groups()

In [ ]:
lfs[-1]['regex']

In [ ]:
from behav_proc import trial_group_cols_all
'|'.join(trial_group_cols_all)

In [ ]:
import gc; gc.collect()

In [ ]:
for pv in df['perturbation'].unique()[1:]:
    dfc = df[(df['tmin'] == '-5.00') & df['subject'].str.startswith('sub01') & \
             (df['perturbation'] == pv)][[f'{kte}_scores','fn_full']]
    fn_full = str( dfc.loc[dfc.index[0], 'fn_full'] )
    print(fn_full)

In [ ]:
f = np.load(fn_full, allow_pickle=1)

In [ ]:
for pv in df['perturbation'].unique()[1:]:
    dpvp =  f['decoding_per_var_and_pert'][()]
    sc = dpvp[('err_sens',pv)]['scores']
    print(scores)

In [ ]:
df['perturbation']

In [ ]:
df[df['tmin'] == '-0.50'][['subject', 'perturbation', f'{kte}_scores']]

In [ ]:
df.groupby(['tmin','perturbation']).mean()['err_sens_scores_mean_over_folds']

# plot_behav2

In [ ]:
script_name = pjoin(scripts_dir,'plot_behav2.py')
ipy.run_line_magic('run', f'-i {script_name}')

# compute_error_sensitivity2

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:
script_name = pjoin(scripts_dir,'compute_error_sensitivity2.py')
ipy.run_line_magic('run', f'-i {script_name}')

In [ ]:
subjects

# plot_corr_decod_es2

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:
script_name = pjoin(scripts_dir,'plot_corr_decod_es2.py')
ipy.run_line_magic('run', f'-i {script_name}')

In [ ]:
#stable_Ridge_spatial_scores_prevmovement_preverrors_errors_prevbelief_broad

# spoc_home_final_fig2

In [ ]:
!which python

In [ ]:
!pip install seaborn

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:
freqs = ['broad', 'theta', 'alpha', 'beta', 'gamma']
script_name = pjoin(scripts_dir,'spoc_home_final_fig2.py')
for hpass in ['no_filter','0.1','0.05']:
    for seed in [0,1]:
        for regression_type in ['Ridge', 'xgboost'][:1]:    
            for freq_name in freqs:
        #for regression_type in ['Ridge']:    
        #for regression_type in ['xgboost']:
                ipy.run_line_magic('run', f'-i {script_name}')
            #print('saved_successfully =', saved_successfully)

print( len(fnames_missing), len(fnames_zero_size) )

In [ ]:
fnames_missing

In [ ]:
print( len(fnames_missing), len(fnames_zero_size) )

# spoc_sliding

In [ ]:
import gc; gc.collect()

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:

time_start = parser.parse('00:00 of 19 jul 2022'); 
time_end = parser.parse('16:00 of 22 sept 2027');   # later at 29 july I occidentally put some err sens decoding files there

freqs = ['broad', 'theta', 'alpha', 'beta', 'gamma']
hpasses = ['no_filter','0.1','0.05']
rts = ['Ridge', 'xgboost']

freqs = ['broad']
#hpasses = ['0.1']
hpasses = ['no_filter']
rts = ['Ridge']
time_lockeds = ['target']
decoding_types = ['classic', 'b2b']
DEBUG=0
script_name = pjoin(scripts_dir,'spoc_sliding2.py')
for hpass in hpasses:
    for regression_type in rts:    
        for freq_name in freqs:
    #for regression_type in ['Ridge']:    
    #for regression_type in ['xgboost']:
            ipy.run_line_magic('run', f'-i {script_name}')
        #print('saved_successfully =', saved_successfully)

In [ ]:
fname_fig

In [ ]:
def listdirPd(folder):
    from datetime import datetime
    fns = os.listdir(folder)
    rows = []
    for f in fns:
        d = {'fname':f, 'folder':folder}
        finfo = os.stat(pjoin(folder,f) )
        d['mtime'] = datetime.fromtimestamp(finfo.st_mtime)
        d['ctime'] = datetime.fromtimestamp(finfo.st_ctime)
        d['atime'] = datetime.fromtimestamp(finfo.st_atime)        
        d['size_bytes'] = finfo.st_size
        d['size_kbytes'] = finfo.st_size / 1024
#         st_mode − protection bits.
#         st_ino − inode number.
#         st_dev − device.
#         st_nlink − number of hard links.
#         st_uid − user id of owner.
#         st_gid − group id of owner.
#         st_size − size of file, in bytes.
#         st_atime − time of most recent access.
#         st_mtime − time of most recent content modification.
#         st_ctime − time of most recent metadata change.
        rows += [d]
    df = pd.DataFrame(rows)
    return df.sort_values(['mtime'])
df_fns =  listdirPd(fld)

In [ ]:
from collections import Counter
Counter( [a.day for a in df_fns['ctime']] )

In [ ]:
fld = pjoin(path_data,subject,'results',output_folder)
fld

In [ ]:
time_locked

In [ ]:
o2 gift card

In [ ]:
df_collect['time_locked']

In [ ]:
df_collect

In [ ]:
(df_collect['env'] == env) & (df_collect['time_locked'] == time_locked)

In [ ]:
time_locked

In [ ]:
sub_df

In [ ]:
sc_aug

In [ ]:
sub_df.columns

In [ ]:
vals.shape

In [ ]:
for (time_locked,decoding_type,env2all_scores) in tpls:

    analysis_name = getAnalysisName(time_locked,control_type)
    var_order = analysis_name2var_ord[analysis_name]

    for vari,varn in enumerate(var_order):
        env2scs = {}
        for env,scs in env2all_scores.items():
            scs_all = np.array(scs)
            tmins_all = scs[-1,:,:]
            tmins_ = tmins_all[0]  # take of the first subject
            scs = scs_all[:-1,:,:]

            vals_ = scs[vari,...]
            #ys_ = vals.mean(axis=0)  # mean over subjects

            # tmins are not sorted
            a = sorted( zip(tmins_,vals_.T), key=lambda x: x[0] )
            tmins,vals = zip(*a)
            vals = np.array(vals).T

            env2scs[env] = np.array(vals)

        # TODO: use argsort to solve
        diff = env2scs['stable'] - env2scs['random']

        plt.title(f'{decoding_type} decoding {varn} differences', fontsize=14)
        # Plot errors during stable environment
        plt.plot(tmins, diff.mean(0), color=colors[0], linewidth=0.5)
        sig = decod_stats(diff) < 0.05
        plt.fill_between(tmins, diff.mean(0), where=sig, color=colors[0], alpha=0.3)
        fname_fig = op.join( path_fig, save_folder,
            f'Exp2{varn}_{regression_type}_{freq_name}_{time_locked}_{decoding_type}' )
        plt.savefig(fname_fig, dpi=400)
        print(f'Fig saved to {fname_fig}')
        plt.close()


In [ ]:
a = sorted( zip(tmins_,vals.T), key=lambda x: x[0] )
tmins,ys = zip(*a)


In [ ]:
ys

In [ ]:
getAnalysisName('target',control_type)

In [ ]:
var_order

In [ ]:
for a in df_collect[['fn_full','tmin']].to_numpy():
    print(a)

In [ ]:

#sub_df['analysis_name']._idx(0)

In [ ]:
np.array( scores_cur_env_cur_subj ).shape

In [ ]:
a = sorted( zip(tmins,ys), key=lambda x: x[0] )
tmins_,ys_ = zip(*a)
plt.plot(tmins,ys, lw=0, marker='*')
plt.plot(tmins_,ys_)

In [ ]:
tmins_all = np.array(scores_cur_env)[...,-1]
np.max( np.std(tmins_all,axis=0) )

In [ ]:
min(tmins),max(tmins)

In [ ]:
tuples_a = np.array(tuples,dtype=object)

In [ ]:
sc

In [ ]:
df_collect

In [ ]:
len( df_collect[ (df_collect['env'] == env) ] )

In [ ]:
sub_df = df_collect[ (df_collect['time_locked'] == time_locked) & (df_collect['env'] == env) ]

In [ ]:
sub_df.reset_index()
print(len(sub_df))
sub_df

In [ ]:
list( sub_df['fn'] ) 

In [ ]:
times, fns, pars = list( zip(*srt) )
from datetime import datetime
days = [ datetime.fromtimestamp(t).day for t in times]
sum( np.array(days) < 20)

In [ ]:

#r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|movement)_(.*)_([a-z])_t=(.*)\.npz', fns[0])
#r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|target)_(.*)_([a-z]).*\.npz', fns[0])
r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|target)_(.*)_(' +freqstr+ ')_t=(.*)\.npz', fns[0])
print(r )
grps = r.groups(); print(grps)
env,rt,tgt,analysis_name,freq_name,tminmax = grps

In [ ]:
len(days)

In [ ]:
fn_full

In [ ]:
shutil.

In [ ]:
# fns[0]
# fn_full = op.join(dir_full,fns[0])
# import shutil
# os.remove(fn_full)

In [ ]:
fn_full

In [ ]:
shift

In [ ]:
np.all( np.diff(times) > 0)

In [ ]:
d = datetime.fromtimestamp(times[0])

In [ ]:
newver = [ 'slide_windows_type' in par_cur for par_cur in pars]

In [ ]:
len(pars) - sum( newver )

In [ ]:
import matplotlib.pyplot as plt
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

plt.hist(days)

In [ ]:
len(srt)

In [ ]:
list ( f.keys() )

In [ ]:
f['par'][()]

In [ ]:
biotec falls,
new datacenter companies fall

In [ ]:
hpass

In [ ]:
len(  env2all_scores[env] )

In [ ]:
# (4, 20, 321) in orig;    so  vars x subj x times

In [ ]:
len(scores_cur_env_cur_subj)

In [ ]:
scs[1,...].shape

In [ ]:
scs.transpose( (2,0,1)).shape

In [ ]:
scs.shape

In [ ]:
from mne.stats import permutation_cluster_1samp_test
help(permutation_cluster_1samp_test)

In [ ]:
len(tminmaxs)

In [ ]:
env2all_scores[env].shape

In [ ]:
print( len(subjects), len(tmins), scs.shape )

In [ ]:
len(env2all_scores['stable'])

In [ ]:
len(env2all_scores['random'])

In [ ]:
scs.shape

In [ ]:
len(tmins)

In [ ]:
env2all_scores

In [ ]:
dd

In [ ]:
os.listdir('/p/project/icei-hbp-2020-0012/lyon/memerr/data2/sub20_YOGCJKKB/results')

In [ ]:
os.listdir(results_folder)

# td_long_fig

In [ ]:
import sys
#for subject in subjects:
#for subject in subjects[:1]:
control_type = 'target'  # 'movement', 'feedback', 'target' or 'belief'
control_type = 'movement'  # 'movement', 'feedback', 'target' or 'belief'
script_name = pjoin(scripts_dir,'td_long_fig2.py')
for hpass in ['no_filter','0.1','0.05']:
    for regression_type in ['Ridge', 'xgboost']:    
        ipy.run_line_magic('run', f'-i {script_name}')
        #sys.exit(1)
        #print('saved_successfully =', saved_successfully)

# td_baselined_final_fig2

In [ ]:
import sys
#for subject in subjects:
#for subject in subjects[:1]:
control_type = 'target'  # 'movement', 'feedback', 'target' or 'belief'
control_type = 'movement'  # 'movement', 'feedback', 'target' or 'belief'
script_name = pjoin(scripts_dir,'td_baselined_final_fig2.py')
for hpass in ['no_filter','0.1','0.05']:
    for regression_type in ['Ridge', 'xgboost']:    
        ipy.run_line_magic('run', f'-i {script_name}')
        sys.exit(1)
        #print('saved_successfully =', saved_successfully)

In [ ]:
path_data

In [ ]:
import gc; gc.collect()

In [ ]:
fnames_missing

In [ ]:
fn = '/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/results/spoc_home2_0.05/stable_xgboost_scores_prevmovement_preverrors_errors_prevbelief_gamma.npy'
f = np.load(fn)

In [ ]:
'sub01_WGPOZPEE spoc_home2_0.05 stable_xgboost_scores_prevmovement_preverrors_errors_prevbelief_gamma.npy',

In [ ]:
f

In [ ]:
fnames_zero_size

In [ ]:
fname_full

In [ ]:
all_scores_stable

# td_long_fig2

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:
script_name = pjoin(scripts_dir,'td_long_fig2.py')
ipy.run_line_magic('run', f'-i {script_name}')